In [1]:
import pandas as pd
import numpy as np
import math
from functools import partial, reduce
import os
import re

In [2]:
dfs = []
input_path = 'C:/Users/lzoeckler/Desktop/5plex/input_data/menzies_raw'
for fname in os.listdir(input_path):
    plex_data = pd.read_csv('{}/{}'.format(input_path, fname),
                            skiprows=13, names=['patient_id', 'type', 'well', 'error',
                                               'HRP2_pg_ml', 'LDH_Pan_pg_ml',
                                               'LDH_Pv_pg_ml', 'LDH_Pf_pg_ml',
                                               'CRP_ng_ml'])
    plex_data = plex_data.applymap(lambda x: x.lower() if isinstance(x, str) else x)
    plex_data['patient_id'] = plex_data['patient_id'].fillna(method='ffill')
    plex_data = plex_data[~plex_data['patient_id'].isnull()]
#     test = plex_data.loc[plex_data['patient_id'].str.contains('pa-001')]
#     if len(test) > 1: 
#         print(fname)
    dfs.append(plex_data)
combined = pd.concat(dfs)
combined = combined.loc[~combined['patient_id'].str.contains('ctrl')]
combined = combined.loc[~combined['type'].isnull()]
combined = combined.loc[~combined['type'].str.contains('replicate')]
combined.head()

,patient_id,type,well,error,HRP2_pg_ml,LDH_Pan_pg_ml,LDH_Pv_pg_ml,LDH_Pf_pg_ml,CRP_ng_ml
15,qdm 1434 2019/03/07 edta wb (neat),reduced concentration,c5,NaN,< 0.68,41.02,28.01,< 5.08,> 38000.00
16,qdm 1434 2019/03/07 edta wb (1:20),reduced concentration,d5,NaN,< 13.60,< 327.20,< 93.80,< 101.60,87928.67
17,qdm 1479 2019/04/10 edta plas (neat),reduced concentration,c4,NaN,5.14,962.33,526.7,15.76,> 38000.00
18,qdm 1479 2019/04/10 edta plas (1:20),reduced concentration,d4,NaN,< 13.60,1056.63,350.9,< 101.60,> 760000.00
19,qem 31 2010/10/19 pfp (neat),reduced concentration,c2,NaN,6.26,30554.78,19174.42,60.81,> 38000.00


In [3]:
def fix_concentrations(df):
    con = df['concentration'].partition(':')[2]
    con = con.partition(')')[0]
    if len(con) != 0:
        return con
    else:
        return '1'

In [4]:
samples_data = combined.copy(deep=True)
samples_data = samples_data.drop('type', axis=1)
samples_data['concentration'] = samples_data['patient_id'].apply(lambda x: x.split(' ')[-1])
samples_data['patient_id'] = samples_data['patient_id'].apply(lambda x: '_'.join(x.split(' ')[:-1]).replace('/', '_'))
samples_data = samples_data.loc[(samples_data['concentration'].str.contains('neat|20'))]
samples_data = samples_data.loc[~samples_data['well'].isnull()]
samples_data['concentration'] = samples_data.apply(fix_concentrations, axis=1)
samples_data = samples_data.sort_values(['patient_id', 'concentration'])
samples_data.head()

,patient_id,well,error,HRP2_pg_ml,LDH_Pan_pg_ml,LDH_Pv_pg_ml,LDH_Pf_pg_ml,CRP_ng_ml,concentration
39,kk_103_2014_04_09_hep_plas,e11,NaN,6.52,66763.83,17747.82,81.37,> 38000.00,1
40,kk_103_2014_04_09_hep_plas,f11,NaN,23.05,82537.55,43492.8,227.52,> 760000.00,20
42,kk_103_2014_04_09_wblamp,g8,NaN,3.02,> 67000.00,19185.53,71.94,> 38000.00,1
43,kk_103_2014_04_09_wblamp,h8,NaN,< 13.60,1563.69,952.99,< 101.60,46420.38,20
41,kk_107_2014_04_15_hep_plas,e12,NaN,2.82,900.77,479.96,22.38,> 38000.00,1


In [5]:
samples_data.loc[samples_data['CRP_ng_ml'].str.contains('<')].head()

,patient_id,well,error,HRP2_pg_ml,LDH_Pan_pg_ml,LDH_Pv_pg_ml,LDH_Pf_pg_ml,CRP_ng_ml,concentration
21,qeck_37_2014_10_16_hep_plas,b11,NaN,< 13.60,< 327.20,109.94,156.6,< 185.60,20
34,qem_150_2011_03_22_pfp,d8,NaN,< 13.60,< 327.20,< 93.80,< 101.60,< 185.60,20
90,qem_205_2011_05_06_pfp,h12,NaN,> 56000.00,106891.68,< 93.80,145979.26,< 185.60,20


In [6]:
samples_data['concentration'].unique()

array(['1', '20'], dtype=object)

In [7]:
sample_ids = samples_data['patient_id'].unique().tolist()
sample_set = set(sample_ids)

In [8]:
# threshold values for various analytes
thresholds = {'ulq': {'HRP2_pg_ml': 2800, 'LDH_Pan_pg_ml': 67000,
                      'LDH_Pv_pg_ml': 19200, 'LDH_Pf_pg_ml': 20800,
                      'CRP_ng_ml': 38000},
              'llq': {'HRP2_pg_ml': .68, 'LDH_Pan_pg_ml': 16.36,
                      'LDH_Pv_pg_ml': 4.96, 'LDH_Pf_pg_ml': 5.08,
                      'CRP_ng_ml': 9.28}}

In [9]:
20*2800

56000

In [10]:
# positivity thresholds for various analytes
pos_thresholds = {'HRP2_pg_ml': 2.3, 'LDH_Pan_pg_ml': 47.8,
                   'LDH_Pv_pg_ml': 75.1, 'CRP_ng_ml': np.nan}

In [11]:
no_duplicates = samples_data.drop_duplicates(subset=['patient_id', 'concentration'], keep=False)
no_duplicates.head()

,patient_id,well,error,HRP2_pg_ml,LDH_Pan_pg_ml,LDH_Pv_pg_ml,LDH_Pf_pg_ml,CRP_ng_ml,concentration
39,kk_103_2014_04_09_hep_plas,e11,NaN,6.52,66763.83,17747.82,81.37,> 38000.00,1
40,kk_103_2014_04_09_hep_plas,f11,NaN,23.05,82537.55,43492.8,227.52,> 760000.00,20
42,kk_103_2014_04_09_wblamp,g8,NaN,3.02,> 67000.00,19185.53,71.94,> 38000.00,1
43,kk_103_2014_04_09_wblamp,h8,NaN,< 13.60,1563.69,952.99,< 101.60,46420.38,20
41,kk_107_2014_04_15_hep_plas,e12,NaN,2.82,900.77,479.96,22.38,> 38000.00,1


In [12]:
for err in no_duplicates['error'].unique():
    print(err)
    print(type(err))

nan
<class 'float'>


In [13]:
def return_decisions(low, high, fail='fail'):
    # Columns = neat: [LLQ, real #, ULQ or within 20% ULQ, NA]
    # Rows = dilution: [LLQ or within 20x LLQ, real #, ULQ, NA]
    other_matrix = np.array([[low, low, fail, fail],
                             [low, low, high, fail],
                             [fail, fail, high, fail],
                             [fail, fail, high, fail]])
    # decisions for various analytes
    decisions = {'HRP2_pg_ml': other_matrix, 'LDH_Pan_pg_ml': other_matrix,
                 'LDH_Pv_pg_ml': other_matrix, 'LDH_Pf_pg_ml': other_matrix,
                 'CRP_ng_ml': other_matrix}
    return(decisions)

In [14]:
def run_compare(df, analyte_val, dil_val):
    # Columns = neat: [LLQ, real #, ULQ or within 20% ULQ, NA]
    # Rows = dilution: [LLQ or within 20x LLQ, real #, ULQ or within 20% ULQ, NA]
    LLQ, real, ULQ, NA = False, False, False, False
    val = df[analyte_val]
    ulq_val = int(dil_val) * thresholds['ulq'][analyte_val]
    llq_val = int(dil_val) * thresholds['llq'][analyte_val]
    try:
        float_val = float(val)
        if math.isnan(float_val):
            NA = True
#         elif float_val > (.99*ulq_val):
#             ULQ = True
        elif (dil_val == '20') and (float_val < 20*llq_val):
            LLQ = True
        elif (dil_val == '1') and (float_val > (.8*ulq_val)):
            ULQ = True
        else:
            real = True
    except ValueError:
        if '<' in val:
            LLQ = True
        elif '>' in val:
            ULQ = True
        else:
            raise ValueError("Unexpected value: {}".format(val))
    finally:
        return(np.array([LLQ, real, ULQ, NA]))

In [15]:
analyte_dfs = []
error_pids = {}
for analyte in thresholds['ulq'].keys():
# for analyte in ['HRP2_pg_ml']:
    print(analyte)
    patient_dfs = []
    for pid in no_duplicates['patient_id'].unique():
#     for pid in ['qem_120_2011_02_19_pfp']:
        patient_data = no_duplicates.loc[no_duplicates['patient_id'] == pid]
        dilution_values = sorted([val for val in patient_data['concentration'].unique() if val != '1'], key=len)
        best_decision = '1'
        for current_dilution in dilution_values:
            best_dil_data = patient_data.loc[patient_data['concentration'].isin([best_decision])]
            current_dil_data = patient_data.loc[patient_data['concentration'].isin([current_dilution])]
#             print('best decision', best_decision)
#             print('current dilution', current_dilution)
            partial_compare_best = partial(run_compare, analyte_val=analyte, dil_val=best_decision)
#             print('partial best', partial_compare_best)
            partial_compare_current = partial(run_compare, analyte_val=analyte, dil_val=current_dilution)
#             print('partial current', partial_compare_current)
            best_dil_data['decision_vector'] = best_dil_data.apply(partial_compare_best, axis=1)
            current_dil_data['decision_vector'] = current_dil_data.apply(partial_compare_current, axis=1)
#             print(current_dil_data['decision_vector'])
            decisions = return_decisions(best_decision, current_dilution)
            decision_matrix = decisions[analyte]
#             print('decision_matrix:', '\n', decision_matrix)
            best_df = pd.DataFrame(columns=['patient_id', 'errors', analyte,
                                            '{}_dilution'.format(analyte),
                                            '{}_well'.format(analyte),
                                            '{}_neat_val'.format(analyte),
                                            '{}_20x_val'.format(analyte)])
            vector_best = best_dil_data.loc[best_dil_data['concentration'] == best_decision,
                                            'decision_vector'].item()
            vector_current = current_dil_data.loc[current_dil_data['concentration'] == current_dilution,
                                                  'decision_vector'].item()
#             print('1 vector', vector_best)
#             print('20 vector', vector_current)
            decision = decision_matrix[vector_current, vector_best].item()
#             print('final decision', decision)
            if decision in [best_decision, current_dilution]:
                if decision == best_decision:
                    dil_data = best_dil_data
                elif decision == current_dilution:
                    dil_data = current_dil_data
                val = dil_data.loc[dil_data['concentration'] == decision,
                                   analyte].item()
                well = dil_data.loc[dil_data['concentration'] == decision,
                                    'well'].item()
                error = dil_data.loc[dil_data['concentration'] == decision,
                                    'error'].item()
                val_for = patient_data.loc[patient_data['concentration'] == '1',
                                           analyte].item()
                val_20_for = patient_data.loc[patient_data['concentration'] == '20',
                                              analyte].item()
            elif decision == 'fail':
                val = 'fail'
                well = 'fail'
                error = np.nan
                val_for = patient_data.loc[patient_data['concentration'] == '1',
                                           analyte].item()
                val_20_for = patient_data.loc[patient_data['concentration'] == '20',
                                              analyte].item()
                try:
                    error_pids[pid] += ', {} failure'.format(analyte)
                except KeyError:
                    error_pids[pid] = '{} failure'.format(analyte)
            else:
                raise ValueError("Unexpected decision value: {}".format(decision))
            other_dilutions = [val for val in patient_data['concentration'].unique()]
            other_dilutions = [float(val) for val in other_dilutions if val not in ('fail', '***')]
            max_dilution = int(max(other_dilutions))
            df_decision = decision if decision not in ('fail', '***') else np.nan
            best_decision = decision
            best_df = best_df.append({'patient_id': pid, 'errors': error, analyte: val,
                                      '{}_dilution'.format(analyte): df_decision,
                                      '{}_well'.format(analyte): well,
                                      '{}_neat_val'.format(analyte): val_for,
                                      '{}_20x_val'.format(analyte): val_20_for}, ignore_index=True)
            if decision == 'fail':
                break
        patient_dfs.append(best_df)
    patient_df = pd.concat(patient_dfs)
    patient_df['errors'] = patient_df['errors'].astype('object')
    analyte_dfs.append(patient_df)
final_df = reduce(lambda left, right: pd.merge(left, right, on=['patient_id', 'errors']), analyte_dfs)
final_df

HRP2_pg_ml


C:\Users\lzoeckler\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\lzoeckler\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\lzoeckler\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:33: FutureWarning: `item` has been deprecated and will be removed in a future version
C:\Users\lzoeckler\AppData\Local\Continuum\anaconda3\

LDH_Pan_pg_ml
LDH_Pv_pg_ml
LDH_Pf_pg_ml
CRP_ng_ml


,patient_id,errors,HRP2_pg_ml,HRP2_pg_ml_dilution,HRP2_pg_ml_well,HRP2_pg_ml_neat_val,HRP2_pg_ml_20x_val,LDH_Pan_pg_ml,LDH_Pan_pg_ml_dilution,LDH_Pan_pg_ml_well,...,LDH_Pf_pg_ml,LDH_Pf_pg_ml_dilution,LDH_Pf_pg_ml_well,LDH_Pf_pg_ml_neat_val,LDH_Pf_pg_ml_20x_val,CRP_ng_ml,CRP_ng_ml_dilution,CRP_ng_ml_well,CRP_ng_ml_neat_val,CRP_ng_ml_20x_val
0,kk_103_2014_04_09_hep_plas,NaN,6.52,1,e11,6.52,23.05,82537.55,20,f11,...,81.37,1,e11,81.37,227.52,> 760000.00,20,f11,> 38000.00,> 760000.00
1,kk_103_2014_04_09_wblamp,NaN,3.02,1,g8,3.02,< 13.60,fail,NaN,fail,...,71.94,1,g8,71.94,< 101.60,46420.38,20,h8,> 38000.00,46420.38
2,kk_107_2014_04_15_hep_plas,NaN,2.82,1,e12,2.82,13.99,900.77,1,e12,...,22.38,1,e12,22.38,156.56,67851.5,20,f12,> 38000.00,67851.5
3,kk_107_2014_04_15_wblamp,NaN,2.12,1,g9,2.12,< 13.60,1563.88,1,g9,...,10.7,1,g9,10.7,385.06,48678.79,20,h9,> 38000.00,48678.79
4,kk_113_2014_05_12_hep_plas,NaN,0.93,1,c11,0.93,< 13.60,218.1,1,c11,...,8.51,1,c11,8.51,142.87,85202.89,20,d11,> 38000.00,85202.89
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,tk_10_2014_04_10_wblamp,NaN,4.99,1,c6,4.99,< 13.60,1431.13,1,c6,...,8.63,1,c6,8.63,< 101.60,50830.62,20,d6,> 38000.00,50830.62
500,tk_3_2014_03_05_hep_plas,NaN,6.85,1,g5,6.85,15.73,251.7,1,g5,...,18.97,1,g5,18.97,< 101.60,81525.53,20,h5,> 38000.00,81525.53
501,tk_3_2014_03_05_wblamp,NaN,5.75,1,c5,5.75,13.64,787.79,1,c5,...,13.46,1,c5,13.46,116.73,108566.69,20,d5,> 38000.00,108566.69
502,tk_7_2014_03_10_hep_plas,NaN,2.48,1,g3,2.48,< 13.60,71.76,1,g3,...,10.37,1,g3,10.37,< 101.60,288182.6,20,h3,> 38000.00,288182.6


In [16]:
# For HRP2 assay only. If Neat value is greater than 100, 20x value is more than
# 10x of neat value or more than ULOQ, set "Alert"

In [17]:
alert_df = final_df.loc[~final_df['HRP2_pg_ml_neat_val'].str.contains('>')]
alert_df = alert_df.loc[~alert_df['HRP2_pg_ml_neat_val'].str.contains('<')]
alert_df = alert_df.loc[~alert_df['HRP2_pg_ml_20x_val'].str.contains('>')]
alert_df = alert_df.loc[~alert_df['HRP2_pg_ml_20x_val'].str.contains('<')]
alert_df = alert_df.loc[alert_df['HRP2_pg_ml'] != 'fail']
alert_df = alert_df.loc[alert_df['HRP2_pg_ml_neat_val'].astype(float) > 100]
alert_df = alert_df.loc[alert_df['HRP2_pg_ml_neat_val'].astype(float) < (.8*2800)]
alert_df = alert_df.loc[alert_df['HRP2_pg_ml_20x_val'].astype(float) < (.8*20*2800)]
alert_ulq = alert_df.loc[alert_df['HRP2_pg_ml_20x_val'].str.contains('>')]
alert_10x = alert_df.loc[~alert_df['HRP2_pg_ml_20x_val'].str.contains('>')]
alert_10x = alert_10x.loc[~alert_10x['HRP2_pg_ml_20x_val'].str.contains('<')]
alert_10x = alert_10x.loc[alert_10x['HRP2_pg_ml_20x_val'].astype(float) > 
                          (alert_10x['HRP2_pg_ml_neat_val'].astype(float) * 8)]
alert_df = pd.concat([alert_ulq, alert_10x])
alert_df['HRP2_pg_ml'] = 'alert'
alert_df['HRP2_pg_ml_dilution'] = np.nan
alert_df['HRP2_pg_ml_well'] = 'alert'
alert_df.head()

,patient_id,errors,HRP2_pg_ml,HRP2_pg_ml_dilution,HRP2_pg_ml_well,HRP2_pg_ml_neat_val,HRP2_pg_ml_20x_val,LDH_Pan_pg_ml,LDH_Pan_pg_ml_dilution,LDH_Pan_pg_ml_well,...,LDH_Pf_pg_ml,LDH_Pf_pg_ml_dilution,LDH_Pf_pg_ml_well,LDH_Pf_pg_ml_neat_val,LDH_Pf_pg_ml_20x_val,CRP_ng_ml,CRP_ng_ml_dilution,CRP_ng_ml_well,CRP_ng_ml_neat_val,CRP_ng_ml_20x_val
363,qem_185_2011_04_29_pfp,NaN,alert,NaN,alert,559.39,4558.25,153.33,1,g4,...,99.35,1,g4,99.35,158.65,64396.87,20,h4,> 38000.00,64396.87
364,qem_185_2011_04_30_pfp,NaN,alert,NaN,alert,155.28,1939.46,98.73,1,e12,...,57.13,1,e12,57.13,294.88,20599.81,20,f12,> 38000.00,20599.81


In [18]:
alert_patients = alert_df['patient_id'].tolist()
final_df = final_df.loc[~final_df['patient_id'].isin(alert_patients)]
final_df = pd.concat([alert_df, final_df])
final_df.head()

,patient_id,errors,HRP2_pg_ml,HRP2_pg_ml_dilution,HRP2_pg_ml_well,HRP2_pg_ml_neat_val,HRP2_pg_ml_20x_val,LDH_Pan_pg_ml,LDH_Pan_pg_ml_dilution,LDH_Pan_pg_ml_well,...,LDH_Pf_pg_ml,LDH_Pf_pg_ml_dilution,LDH_Pf_pg_ml_well,LDH_Pf_pg_ml_neat_val,LDH_Pf_pg_ml_20x_val,CRP_ng_ml,CRP_ng_ml_dilution,CRP_ng_ml_well,CRP_ng_ml_neat_val,CRP_ng_ml_20x_val
363,qem_185_2011_04_29_pfp,NaN,alert,NaN,alert,559.39,4558.25,153.33,1,g4,...,99.35,1,g4,99.35,158.65,64396.87,20,h4,> 38000.00,64396.87
364,qem_185_2011_04_30_pfp,NaN,alert,NaN,alert,155.28,1939.46,98.73,1,e12,...,57.13,1,e12,57.13,294.88,20599.81,20,f12,> 38000.00,20599.81
0,kk_103_2014_04_09_hep_plas,NaN,6.52,1,e11,6.52,23.05,82537.55,20,f11,...,81.37,1,e11,81.37,227.52,> 760000.00,20,f11,> 38000.00,> 760000.00
1,kk_103_2014_04_09_wblamp,NaN,3.02,1,g8,3.02,< 13.60,fail,NaN,fail,...,71.94,1,g8,71.94,< 101.60,46420.38,20,h8,> 38000.00,46420.38
2,kk_107_2014_04_15_hep_plas,NaN,2.82,1,e12,2.82,13.99,900.77,1,e12,...,22.38,1,e12,22.38,156.56,67851.5,20,f12,> 38000.00,67851.5


In [19]:
super_final = final_df.copy(deep=True)
for pid in error_pids.keys():
    error = error_pids[pid]
    pid_df = final_df.loc[final_df['patient_id'] == pid]
    pid_df['errors'] = pid_df['errors'].apply(lambda x: error if np.isnan(x) else x + ' ' + error)
    if len(pid_df) > 0:
        super_final = super_final.loc[super_final['patient_id'] != pid]
        super_final = super_final.append(pid_df)
print(len(super_final))
super_final.head()

504


C:\Users\lzoeckler\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,patient_id,errors,HRP2_pg_ml,HRP2_pg_ml_dilution,HRP2_pg_ml_well,HRP2_pg_ml_neat_val,HRP2_pg_ml_20x_val,LDH_Pan_pg_ml,LDH_Pan_pg_ml_dilution,LDH_Pan_pg_ml_well,...,LDH_Pf_pg_ml,LDH_Pf_pg_ml_dilution,LDH_Pf_pg_ml_well,LDH_Pf_pg_ml_neat_val,LDH_Pf_pg_ml_20x_val,CRP_ng_ml,CRP_ng_ml_dilution,CRP_ng_ml_well,CRP_ng_ml_neat_val,CRP_ng_ml_20x_val
363,qem_185_2011_04_29_pfp,NaN,alert,NaN,alert,559.39,4558.25,153.33,1,g4,...,99.35,1,g4,99.35,158.65,64396.87,20,h4,> 38000.00,64396.87
364,qem_185_2011_04_30_pfp,NaN,alert,NaN,alert,155.28,1939.46,98.73,1,e12,...,57.13,1,e12,57.13,294.88,20599.81,20,f12,> 38000.00,20599.81
0,kk_103_2014_04_09_hep_plas,NaN,6.52,1,e11,6.52,23.05,82537.55,20,f11,...,81.37,1,e11,81.37,227.52,> 760000.00,20,f11,> 38000.00,> 760000.00
2,kk_107_2014_04_15_hep_plas,NaN,2.82,1,e12,2.82,13.99,900.77,1,e12,...,22.38,1,e12,22.38,156.56,67851.5,20,f12,> 38000.00,67851.5
3,kk_107_2014_04_15_wblamp,NaN,2.12,1,g9,2.12,< 13.60,1563.88,1,g9,...,10.7,1,g9,10.7,385.06,48678.79,20,h9,> 38000.00,48678.79


In [20]:
super_final['errors'].unique()

array([nan, 'HRP2_pg_ml failure',
       'HRP2_pg_ml failure, LDH_Pf_pg_ml failure',
       'LDH_Pan_pg_ml failure, LDH_Pv_pg_ml failure', 'CRP_ng_ml failure'],
      dtype=object)

In [21]:
# final_ids = super_final['patient_id'].unique().tolist()
# final_set = set(final_ids)
# assert final_set == sample_set, print(sample_set - final_set)

In [22]:
def split_time(df, run):
    if run:
        sub = df['patient_id'].split('_')
        try:
            time = '_'.join(sub[2:])
            return time
        except IndexError:
            return 0
    else:
        raise KeyError('BUSTOOOO')

In [23]:
def remove_time(df):
    patient = df['patient_id'].split('_')
    return '_'.join(patient[0:2])

In [24]:
time_df = super_final.copy(deep=True)
time_df.sort_values(['patient_id'], inplace=True)
time_df.set_index(['patient_id'], inplace=True)
time_df.head()

,errors,HRP2_pg_ml,HRP2_pg_ml_dilution,HRP2_pg_ml_well,HRP2_pg_ml_neat_val,HRP2_pg_ml_20x_val,LDH_Pan_pg_ml,LDH_Pan_pg_ml_dilution,LDH_Pan_pg_ml_well,LDH_Pan_pg_ml_neat_val,...,LDH_Pf_pg_ml,LDH_Pf_pg_ml_dilution,LDH_Pf_pg_ml_well,LDH_Pf_pg_ml_neat_val,LDH_Pf_pg_ml_20x_val,CRP_ng_ml,CRP_ng_ml_dilution,CRP_ng_ml_well,CRP_ng_ml_neat_val,CRP_ng_ml_20x_val
patient_id,,,,,,,,,,,,,,,,,,,,,
kk_103_2014_04_09_hep_plas,NaN,6.52,1,e11,6.52,23.05,82537.55,20,f11,66763.83,...,81.37,1,e11,81.37,227.52,> 760000.00,20,f11,> 38000.00,> 760000.00
kk_103_2014_04_09_wblamp,"LDH_Pan_pg_ml failure, LDH_Pv_pg_ml failure",3.02,1,g8,3.02,< 13.60,fail,NaN,fail,> 67000.00,...,71.94,1,g8,71.94,< 101.60,46420.38,20,h8,> 38000.00,46420.38
kk_107_2014_04_15_hep_plas,NaN,2.82,1,e12,2.82,13.99,900.77,1,e12,900.77,...,22.38,1,e12,22.38,156.56,67851.5,20,f12,> 38000.00,67851.5
kk_107_2014_04_15_wblamp,NaN,2.12,1,g9,2.12,< 13.60,1563.88,1,g9,1563.88,...,10.7,1,g9,10.7,385.06,48678.79,20,h9,> 38000.00,48678.79
kk_113_2014_05_12_hep_plas,NaN,0.93,1,c11,0.93,< 13.60,218.1,1,c11,218.1,...,8.51,1,c11,8.51,142.87,85202.89,20,d11,> 38000.00,85202.89


In [25]:
time_df.to_csv('C:/Users/lzoeckler/Desktop/3_5_dilutions.csv')

In [69]:
partial_format = samples_data.copy(deep=True)
# partial_format['time_point_days'] = partial_format.apply(split_time, axis=1)
# partial_format['patient_id'] = partial_format.apply(remove_time, axis=1)
# partial_format = partial_format.merge(add_info, how='left', on=['patient_id', 'time_point_days'])
partial_format.sort_values(['patient_id'], inplace=True)
partial_format.set_index(['patient_id'], inplace=True)
partial_format.head()

,well,error,HRP2_pg_ml,LDH_Pan_pg_ml,LDH_Pv_pg_ml,LDH_Pf_pg_ml,CRP_ng_ml,concentration
patient_id,,,,,,,,
kk_103_2014_04_09_hep_plas,e11,NaN,6.52,66763.83,17747.82,81.37,> 38000.00,1
kk_103_2014_04_09_hep_plas,f11,NaN,23.05,82537.55,43492.8,227.52,> 760000.00,20
kk_103_2014_04_09_wblamp,g8,NaN,3.02,> 67000.00,19185.53,71.94,> 38000.00,1
kk_103_2014_04_09_wblamp,h8,NaN,< 13.60,1563.69,952.99,< 101.60,46420.38,20
kk_107_2014_04_15_hep_plas,e12,NaN,2.82,900.77,479.96,22.38,> 38000.00,1


In [71]:
partial_format.to_csv('C:/Users/lzoeckler/Desktop/partially_formatted.csv')